# Test Case LVV-T1598

This test case is to support the re-verification of the functional requirements of the camera hexapod's hardware.
The actual tests are done with a laser tracker and mechanical gauges. 
This notebook is to do the movements and record the timestamp when a movement was done.

Requirements
* EFD
* Hexapod(s)powered on
* Thermal sensors attached to the six actuators of the hexapod.
* CSC running
* KAFKA producer running
* Laser tracker set up.

This test will require manual verification of certain events and telemetry in the summit EFD.
Also manual verification of appropriate temperatures for each actuator.

In [1]:
from lsst.ts import salobj
from lsst.ts.idl.enums import MTHexapod
import logging
import asyncio
import os
import yaml

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd
import astropy.units as u

from lsst_efd_client import EfdClient

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-0.4.1/lib/python3.8/site-packages/jose/backends/cryptography_backend.py:23: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes, int_to_bytes


In [2]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])

file:///home/hdrass/WORK/ts_ddsconfig/config/ospl-sp.xml
summit


In [3]:
#This step should fail at NCSA since there is only one domain
print(os.environ["LSST_DDS_DOMAIN_ID"])

11


In [4]:
print(os.environ["LSST_DDS_INTERFACE"])

net1


The following block sets the necessary environment variables for setting up the DDS/SAL communication

In [5]:
#Wait for history data [sec] default value is 60
#os.environ["LSST_DDS_DOMAIN"] = 'citest'
#os.environ["LSST_DDS_HISTORYSYNC"] = "10"

In [6]:
STD_WAIT = 39 # This is the amount of time to wait in between a move just to make sure that the actuators are not overheating

In [7]:
log = logging.getLogger(__name__) # This sets up the logger for the test.
#logging.setLevel(20)
logging.basicConfig(level=logging.INFO)


In [8]:
start_time = datetime.now()
test_message = "Camera Hexapod Integration Test"
#What means index=1 here?
#script = salobj.Controller("Script", index=1)
script = salobj.Controller("Script", index=42658885)
await asyncio.sleep(10) #wait 10 second may help with DDS problems; closing all other kernels may help too
print(f'time to start is {datetime.now() - start_time} [s]')

INFO:Script:Read historical data in 0.00 sec


time to start is 0:00:12.010365 [s]


In [9]:
await script.start_task

This is how you start the remote for the CSC.

In [11]:
# create the remote
#index=1 is the camera hexapod!! index=2 is the M2 hexapod!
csc_index = 1
hexapod_csc = salobj.Remote(name="MTHexapod", domain=script.domain, index=csc_index)

INFO:MTHexapod:Read historical data in 0.00 sec


In [12]:
await hexapod_csc.start_task

In [13]:
await hexapod_csc.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
await salobj.set_summary_state(hexapod_csc, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(hexapod_csc, salobj.State.OFFLINE)

In [ ]:
await salobj.set_summary_state(hexapod_csc, salobj.State.DISABLED)

In [ ]:
#This is to bring the state machine in the right starting state
await salobj.set_summary_state(hexapod_csc, salobj.State.ENABLED)

In [14]:
print(hexapod_csc)
#Bo:Check which hexapod we are controlling
whichHex1 = await hexapod_csc.evt_configuration.aget(timeout=10.)
print("Hexapod hex", whichHex1.MTHexapodID)
print(hexapod_csc.salinfo.index)

Hexapod hex 1
1


In [15]:
#This cell makes sure that the data go to EFD
#may need to wait a few seconds before event shows up in EFD
#client = EfdClient('ncsa_teststand_efd')
client = EfdClient('summit_efd')

In [16]:
end = Time(datetime.now())
start = end - timedelta(hours=3)
dfe = await client.select_time_series('lsst.sal.MTHexapod.logevent_summaryState', '*', start.tai, end.tai, csc_index)

In [17]:
dfe

,MTHexapodID,priority,private_host,private_identity,private_kafkaStamp,private_origin,private_rcvStamp,private_revCode,private_seqNum,private_sndStamp,summaryState
2021-03-08 15:48:30.196000+00:00,1,0,0,MTHexapod:1,1.615219e+09,5360,1.615219e+09,959a0a03,12,1.615219e+09,3
2021-03-08 15:49:32.600000+00:00,1,0,0,MTHexapod:1,1.615219e+09,5360,1.615219e+09,959a0a03,13,1.615219e+09,5
2021-03-08 15:49:41.069000+00:00,1,0,0,MTHexapod:1,1.615219e+09,5360,1.615219e+09,959a0a03,14,1.615219e+09,1
2021-03-08 15:49:50.107000+00:00,1,0,0,MTHexapod:1,1.615219e+09,5360,1.615219e+09,959a0a03,15,1.615219e+09,2


In [18]:
def enabled_substate_callback(evt):
    """Print the enabled substate when event is received."""
    print(MTHexapod.EnabledSubstate(evt.enabledSubstate))

In [19]:
def in_position_callback(evt):
    """Print the in position event when it is received."""
    print(evt.in_position)

This next block will make sure that the CSC is ready for the test

In [31]:
connected = await hexapod_csc.evt_connected.aget() # This is how you get an event/telemetry with await remote.type_name.aget() type being [evt,tel] and name being the name of the topic
#commandable = connected.command
telemetry_working = connected.telemetry

if not connected or not telemetry_working:
    raise Exception("Hexapod not connected or telemetry not being received.") 

In [32]:
controller_state = await hexapod_csc.evt_controllerState.aget()
state = controller_state.controllerState
offline_substate = controller_state.offlineSubstate
print(state)
#This is only for the real system. The simulater only has the AVAILABLE offline state 

'''  
if not state == salobj.State.OFFLINE and not offline_substate == MTHexapod.OfflineSubstate.AVAILABLE:
    raise Exception("Controller must be changed to Available Offline Substate.")
'''
commandable_by_dds = await hexapod_csc.evt_commandableByDDS.aget()
dds_state = commandable_by_dds.state
if not dds_state:
    raise Exception("Controller must in CommandableByDDS state.")

2


The next four blocks bring the CSC to the Enabled state.

In [ ]:
await hexapod_csc.cmd_enterControl.set_start() # Send the enterControl command notice the casing and the set_start method

In [ ]:
await hexapod_csc.cmd_start.set_start() 

In [ ]:
await hexapod_csc.cmd_enable.set_start()

In [ ]:
await hexapod_csc.cmd_clearError.set_start() # This clears the error

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [ ]:
# Step skiped see deviation LVV-T1802 Test Step 5                                  
#hexapod_csc.evt_controllerState.callback = enabled_substate_callback                                
#await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=5900, u=0.12,v=0,w=0,sync=True)

In [ ]:
#Command used to stop the hexapod 
await hexapod_csc.cmd_stop.set_start()

In [36]:
#This command is to set the Hexapod to zero position
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

In [37]:
#This command is to move the Hexapod to any position you like
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1598 functional Test - Test move - Starting time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=5000,y=0,z=0, u=0,v=0,w=0,sync=True)

INFO:Script:START- Camera Hexapod Integration Test -- LVV-T1598 functional Test - Test move - Starting time: 2021-03-08 18:50:44.138441 UTC


In [22]:
script.log.info(f"TEST to see if annotations like this arrive in the EFD.")

INFO:Script:TEST to see if annotations like this arrive in the EFD.


In [ ]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1598 functional Test - Step 1 - Starting time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=5900,u=0,v=0.12,w=0,sync=True)
script.log.info(f"STOP- {test_message} -- LVV-T1598 functional Test - Step 1 ")
await asyncio.sleep(STD_WAIT)

In [ ]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1598 functional Test - Step 2 - Starting time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=5900,u=0,v=-0.12,w=0,sync=True)
script.log.info(f"STOP- {test_message} -- LVV-T1598 functional Test - Step 2 ")
await asyncio.sleep(STD_WAIT)

In [ ]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1598 functional Test - Step 3 - Starting time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=5900,u=0.12,v=0,w=0,sync=True)
script.log.info(f"STOP- {test_message} -- LVV-T1598 functional Test - Step 3 ")
await asyncio.sleep(STD_WAIT)

In [ ]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1598 functional Test - Step 4 - Starting time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=5900,u=-0.12,v=0,w=0,sync=True)
script.log.info(f"STOP- {test_message} -- LVV-T1598 functional Test - Step 4 ")
await asyncio.sleep(STD_WAIT)

In [ ]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1598 functional Test - Step 5 - Starting time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=-5900,u=0,v=0.12,w=0,sync=True)
script.log.info(f"STOP- {test_message} -- LVV-T1598 functional Test - Step 5 ")
await asyncio.sleep(STD_WAIT)

In [ ]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1598 functional Test - Step 6 - Starting time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=-5900,u=0,v=-0.12,w=0,sync=True)
script.log.info(f"STOP- {test_message} -- LVV-T1598 functional Test - Step 6 ")
await asyncio.sleep(STD_WAIT)

In [ ]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1598 functional Test - Step 7 - Starting time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=-5900,u=0.12,v=0,w=0,sync=True)
script.log.info(f"STOP- {test_message} -- LVV-T1598 functional Test - Step 7 ")
await asyncio.sleep(STD_WAIT)

In [ ]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1598 functional Test - Step 8 - Starting time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=-5900,u=-0.12,v=0,w=0,sync=True)
script.log.info(f"STOP- {test_message} -- LVV-T1598 functional Test - Step 8 ")
await asyncio.sleep(STD_WAIT)

In [ ]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1598 functional Test - Step 9 - Starting time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=5900,u=0,v=0.12,w=0,sync=True)
script.log.info(f"STOP- {test_message} -- LVV-T1598 functional Test - Step 9 ")
await asyncio.sleep(STD_WAIT)

In [ ]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1598 functional Test - Step 10 - Starting time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=5900,u=0,v=-0.12,w=0,sync=True)
script.log.info(f"STOP- {test_message} -- LVV-T1598 functional Test - Step 10 ")
await asyncio.sleep(STD_WAIT)

In [ ]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1598 functional Test - Step 11 - Starting time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=5900,u=0.12,v=0,w=0,sync=True)
script.log.info(f"STOP- {test_message} -- LVV-T1598 functional Test - Step 11 ")
await asyncio.sleep(STD_WAIT)

In [ ]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1598 functional Test - Step 12 - Starting time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=5900,u=-0.12,v=0,w=0,sync=True)
script.log.info(f"STOP- {test_message} -- LVV-T1598 functional Test - Step 12 ")
await asyncio.sleep(STD_WAIT)

In [ ]:
await salobj.set_summary_state(hexapod_csc, salobj.State.STANDBY) # Transition the CSC to Standby state

In [ ]:
await domain.close() # Close the remote connection